In [8]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D,Dropout, Reshape, LSTM, BatchNormalization,TimeDistributed
from keras.layers import Conv2D
from keras.optimizers import Adam #, RMSprop, SGD
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from time import time

import pickle
import random
import numpy as np
import sys
import h5py
import glob
import os

Using TensorFlow backend.


In [9]:
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 16
rows, cols = 431, 128   
nb_epoch = 200
pool_size = (5,5)                  # size of pooling area for max pooling
prob_drop_conv = 0.3                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.3              # drop probability for dropout @ fc layer

In [10]:
#might need to rewrite these



def fileLists():
    trainlist=[]
    validationlist=[]
    testlist=[]
    evalSetupFiles='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\evaluation_setup\\*.txt'
    txtfilelist=glob.glob(evalSetupFiles)
    for txt in txtfilelist:
        if '_location' not in txt:
            continue
        with open(txt,'r') as evaltxtfile:
            for line in evaltxtfile.readlines():
                line=line.strip().split('\t')[0]
                if 'train' in txt:
                    trainlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                elif 'test' in txt:
                    testlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                else:
                    validationlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
    print('trainfiles: ', str(len(trainlist)))
    print('validationfiles: ', str(len(validationlist)))
    print('testfiles: ', str(len(testlist)))
    return trainlist,validationlist,testlist

labelRef={'barcelona':0, 'helsinki':1, 'london':2, 'paris':3, 'stockholm':4, 'vienna':5, 
          'airport':6, 'shopping_mall':7,'metro_station':8,'street_pedestrian':9,'public_square':10,
          'street_traffic':11,'tram':12,'bus':13,'metro':14,'park':15}

def getData(flist):
    pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
    X_=np.zeros(((len(flist)),128,431))
    Y_=np.zeros(((len(flist)),nb_classes))
    for i,tfile in enumerate(flist):
        with open(pth+tfile, "rb" ) as scenesample:
            fv=pickle.load(scenesample)
        X_[i,:,:]=fv
        location=tfile.split('-')[1]
        Y_[i,labelRef[location]]=1
        scene=tfile.split('-')[0].replace('logMelSpec\\','')
        Y_[i,labelRef[scene]]=1
    return X_, Y_
#trainlist,validationlist,testlist=fileLists()
#X_train, Y_train = getData(trainlist)
#X_val, Y_val = getData(validationlist)
#X_test, Y_test = getData(testlist)

print(' data sorted')


 data sorted


In [11]:
###### Convolutional model
def compileCRNN(cols,rows,nb_classes=1):

    model = Sequential()
    # conv1 layer
    model.add(Conv2D(32, (7, 7), padding='same', activation='relu', input_shape=(cols,rows,1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv2 layer
    model.add(Conv2D(64, (7, 7), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(4,7), strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv3 layer
    model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(prob_drop_conv))

    model.add(Flatten())
    # fc1 layer
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(prob_drop_hidden))
    model.add(BatchNormalization())
   
    # fc2 layer
    model.add(Dense(nb_classes, activation='sigmoid'))
    opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
   
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_accuracy'])
    model.summary()
    return model

model = compileCRNN(cols,rows,nb_classes=nb_classes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 431, 32)      1600      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 431, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 216, 32)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 216, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 216, 64)       100416    
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 216, 64)       256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 108, 64)       0         
__________

In [12]:
# Train
#fold=1
def train_network(model, model_name, X_train, Y_train, X_val, Y_val, nb_epoch, validationsplit_size, batchsize, early_stoping_patience, output_folder):
    checkpointer = ModelCheckpoint(filepath=os.path.join(output_folder,model_name + '.hdf5'),save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=early_stoping_patience)
    tensorboard = TensorBoard(log_dir="logs\\{}".format(time()))
    Callbacks=[checkpointer,  tensorboard] #early_stopping,
  #  print(samples)
    steps=int(samples/batchsize)
    validationsteps=int(validationsplit_size/batchsize)
    history = model.fit(X_train, Y_train, epochs=nb_epoch, callbacks=Callbacks, batch_size=batch_size, validation_data=(X_val, Y_val), shuffle=True, verbose=1)
    return history,model

def buildModel(savemodelfilename, samples,model,X_train, Y_train,X_val, Y_val):
    valSplit_size = int(samples/4)
    early_stoping_patience=5
    history,model = train_network(model, 'models\\multi_label', X_train, Y_train, X_val, Y_val, nb_epoch, valSplit_size, batch_size, early_stoping_patience,'.')
    model.save_weights(savemodelfilename)
    return model, history


In [13]:
from datetime import datetime
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 16
rows, cols = 431, 128  

trainlist,validationlist,testlist=fileLists()
print('lists sorted')
X_tr, Y_train = getData(trainlist)
X_v, Y_val = getData(validationlist)
X_te, Y_test = getData(testlist)
print('data obtained')
X_train=np.expand_dims(X_tr,axis=3)
X_val=np.expand_dims(X_v,axis=3)
X_test=np.expand_dims(X_te,axis=3)
print(X_train.shape, X_val.shape, X_test.shape)
print('data sorted')
model = compileCRNN(cols,rows,nb_classes=nb_classes)
print('model compiled')
savemodelfilename='models\\multi_label.testsave'
samples=(X_train.shape[0])
print(datetime.now())
models,histories= buildModel(savemodelfilename, samples,model,X_train,Y_train,X_val, Y_val)
print(datetime.now())
print('model saved')

trainfiles:  6122
validationfiles:  2518
testfiles:  2518
lists sorted
data obtained
(6122, 128, 431, 1) (2518, 128, 431, 1) (2518, 128, 431, 1)
data sorted
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 128, 431, 32)      1600      
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 431, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 216, 32)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 64, 216, 32)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 216, 64)       100416    
_________________________________________________________________
batch_normalization_6 (Batch (None, 64, 216, 64)   

6122/6122 [==============================] - 23s 4ms/step - loss: 0.2602 - binary_accuracy: 0.8940 - val_loss: 3.1545 - val_binary_accuracy: 0.7604
Epoch 35/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.2596 - binary_accuracy: 0.8948 - val_loss: 3.7358 - val_binary_accuracy: 0.6870
Epoch 36/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.2572 - binary_accuracy: 0.8965 - val_loss: 5.6910 - val_binary_accuracy: 0.5406
Epoch 37/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.2529 - binary_accuracy: 0.8977 - val_loss: 9.5262 - val_binary_accuracy: 0.3444
Epoch 38/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.2504 - binary_accuracy: 0.8989 - val_loss: 3.6583 - val_binary_accuracy: 0.6438
Epoch 39/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.2446 - binary_accuracy: 0.9018 - val_loss: 3.8762 - val_binary_accuracy: 0.6770
Epoch 40/200
6122/6122 [=======================

Epoch 85/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1714 - binary_accuracy: 0.9315 - val_loss: 2.7936 - val_binary_accuracy: 0.7155
Epoch 86/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.1786 - binary_accuracy: 0.9287 - val_loss: 0.4444 - val_binary_accuracy: 0.8549
Epoch 87/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1734 - binary_accuracy: 0.9306 - val_loss: 0.7962 - val_binary_accuracy: 0.8481
Epoch 88/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1759 - binary_accuracy: 0.9285 - val_loss: 0.3877 - val_binary_accuracy: 0.8668
Epoch 89/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1760 - binary_accuracy: 0.9293 - val_loss: 0.6203 - val_binary_accuracy: 0.8469
Epoch 90/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1764 - binary_accuracy: 0.9276 - val_loss: 0.6071 - val_binary_accuracy: 0.8553
Epoch 91/200
6122/6122 [==========

6122/6122 [==============================] - 23s 4ms/step - loss: 0.1259 - binary_accuracy: 0.9506 - val_loss: 0.3667 - val_binary_accuracy: 0.8803
Epoch 136/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1329 - binary_accuracy: 0.9461 - val_loss: 0.7033 - val_binary_accuracy: 0.8660
Epoch 137/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.1267 - binary_accuracy: 0.9494 - val_loss: 2.0619 - val_binary_accuracy: 0.7859
Epoch 138/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1259 - binary_accuracy: 0.9499 - val_loss: 1.2337 - val_binary_accuracy: 0.8346
Epoch 139/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1242 - binary_accuracy: 0.9504 - val_loss: 0.9331 - val_binary_accuracy: 0.8506
Epoch 140/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1264 - binary_accuracy: 0.9506 - val_loss: 0.5385 - val_binary_accuracy: 0.8767
Epoch 141/200
6122/6122 [=================

6122/6122 [==============================] - 23s 4ms/step - loss: 0.1257 - binary_accuracy: 0.9501 - val_loss: 0.8124 - val_binary_accuracy: 0.8701
Epoch 186/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1173 - binary_accuracy: 0.9529 - val_loss: 0.5273 - val_binary_accuracy: 0.8830
Epoch 187/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1139 - binary_accuracy: 0.9543 - val_loss: 0.4145 - val_binary_accuracy: 0.8908
Epoch 188/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1073 - binary_accuracy: 0.9576 - val_loss: 0.4118 - val_binary_accuracy: 0.8938
Epoch 189/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1012 - binary_accuracy: 0.9602 - val_loss: 0.3614 - val_binary_accuracy: 0.8927
Epoch 190/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1000 - binary_accuracy: 0.9609 - val_loss: 0.3247 - val_binary_accuracy: 0.8933
Epoch 191/200
6122/6122 [=================